up vote
3
down vote
favorite

I want to retrieve all lat/lon coordinate pairs of a regular grid over a certain map area. I have found the geopy library, but didn't manage at all to approach the problem.

For example, I have a rectangular geographic area described by its four corners in lat/lon coordinates, I seek to calculate the grid with spacing of e.g. 1km covering this area.
python google-maps geopy
shareedit
edited Oct 31 '16 at 15:05
asked Oct 31 '16 at 12:38
Amira Akhdal
2614

    1
    Cardinal points like NSEW? How can cardinal points be given in (lon, lat)-pairs and how can they have a distance? Please elaborate more on your problem and add the code you have so far. – jbndlr Oct 31 '16 at 12:47
    Sorry, i was wrong i want to find all lat long coordinates in a certain area, for example i have 4 points lat long of a region in google maps, i want to retrive all coordinates lat long in this area with 10 kilometers of distance each other. i dont have a code now, im still searching something in geopy. – Amira Akhdal Oct 31 '16 at 12:54
    So you want to create a regular grid of point coordinates given a certain polygon? – jbndlr Oct 31 '16 at 12:56
    yes, is this! is there anything in geopy or in other library ? – Amira Akhdal Oct 31 '16 at 12:58

add a comment
1 Answer
active
oldest
votes
up vote
9
down vote
accepted
Preliminary Considerations

It makes a slight difference how your certain area is defined. If it's just a rectangular area (note: rectangular in projection is not neccessarily rectangular on Earth's surface!), you can simply iterate from min to max in both coordinate dimensions using your desired step size. If you have an arbitrary polygon shape at hand, you need to test which of your generated points intersects with that poly and only return those coordinate pairs for which this condition holds true.
Calculating a Regular Grid

A regular grid does not equal a regular grid across projections. You are talking about latitude/longitude pairs, which is a polar coordinate system measured in degrees on an approximation of Earth's surface shape. In lat/lon (EPSG:4326), distances are not measured in meters/kilometers/miles, but rather in degrees.

Furthermore, I assume you want to calculate a grid with its "horizontal" steps being parallel to the equator (i.e. latitudes). For other grids (for example rotated rectangular grids, verticals being parallel to longitudes, etc.) you need to spend more effort transforming your shapes.

Ask yourself: Do you want to create a regularly spaced grid in degrees or in meters?

A grid in degrees

If you want it in degrees, you can simply iterate:

stepsize = 0.001
for x in range(lonmin, lonmax, stepsize):
    for y in range(latmin, latmax, stepsize):
        yield (x, y)

But: Be sure to know that the length in meters of a step in degrees is not the same across Earth's surface. For example, 0.001 delta degrees of latitude close to the equator covers a different distance in meters on the surface than it does close to the poles.

A grid in meters

If you want it in meters, you need to project your lat/lon boundaries of your input area (your certain area on a map) into a coordinate system that supports distances in meters. You can use the Haversine formula as a rough approximation to calculate the distance between lat/lon pairs, but this is not the best method you can use.

Better yet is to search for a suitable projection, transform your area of interest into that projection, create a grid by straightforward iteration, get the points, and project them back to lat/lon pairs. For example, a suitable projection for Europe would be EPSG:3035. Google Maps uses EPSG:900913 for their web mapping service, by the way.

In python, you can use the libraries shapely and pyproj to work on geographic shapes and projections:

In [31]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import shapely.geometry
import pyproj
from pyproj import Proj

In [37]:
# Set up projections

p2 = Proj('+proj=utm +zone=6 +ellps=WGS84') # use proj4 string
x2,y2 = p2(-84.6125, 39.157778)
'x2=%9.3f y2=%11.3f' % (x2,y2)
#'x=765975.641 y=3805993.134'

p3 = Proj('+proj=utm +zone=6 +ellps=WGS84') # use proj4 string
x3,y3 = p3(-84.512016, 39.103119)
'x3=%9.3f y3=%11.3f' % (x3,y3)



'x2=5877034.686 y2=6699041.776'

'x3=5891085.037 y3=6702959.002'

In [30]:

# cheviot : 39.157778, -84.6125
# cincinnati :  39.103119, -84.512016

p_ll = Proj('+proj=utm +zone=10 +ellps=WGS84')
p_mt = Proj(init='epsg:3857')

# Create corners of rectangle to be transformed to a grid
nw = shapely.geometry.Point((-84.6125, 39.157778))
se = shapely.geometry.Point((-84.512016, 39.103119))
stepsize = 200 # 200 m grid step size

In [24]:
p_ll = pyproj.Proj(init='epsg:4326')
p_mt = pyproj.Proj(init='epsg:3857') # metric; same as EPSG:900913
nw = shapely.geometry.Point((-5.0, 40.0))
se = shapely.geometry.Point((-4.0, 41.0))
stepsize = 5000 # 100 m grid step size

In [25]:
# Project corners to target projection
s = pyproj.transform(p_ll, p_mt, nw.x, nw.y) # Transform NW point to 3857
e = pyproj.transform(p_ll, p_mt, se.x, se.y) # .. same for SE

s
e

(-556597.4539663672, 4865942.279503176)

(-445277.9631730949, 5012341.663847514)

In [19]:
# Iterate over 2D area
gridpoints = []
x = s[0]
while x < e[0]:
    y = s[1]
    while y < e[1]:
        p = shapely.geometry.Point(pyproj.transform(p_mt, p_ll, x, y))
        print(p)
        gridpoints.append(p)
        y += stepsize
    x += stepsize

#gridpoints

POINT (-4.999999999999994 40)
POINT (-4.999999999999994 40.0343988020191)
POINT (-4.999999999999994 40.06878026248516)
POINT (-4.999999999999994 40.10314437776663)
POINT (-4.999999999999994 40.1374911442588)
POINT (-4.999999999999994 40.17182055838371)
POINT (-4.999999999999994 40.20613261659013)
POINT (-4.999999999999994 40.24042731535354)
POINT (-4.999999999999994 40.27470465117607)
POINT (-4.999999999999994 40.3089646205865)
POINT (-4.999999999999994 40.34320722014021)
POINT (-4.999999999999994 40.37743244641913)
POINT (-4.999999999999994 40.41164029603176)
POINT (-4.999999999999994 40.44583076561306)
POINT (-4.999999999999994 40.48000385182451)
POINT (-4.999999999999994 40.51415955135401)
POINT (-4.999999999999994 40.54829786091585)
POINT (-4.999999999999994 40.5824187772507)
POINT (-4.999999999999994 40.61652229712559)
POINT (-4.999999999999994 40.65060841733385)
POINT (-4.999999999999994 40.68467713469507)
POINT (-4.999999999999994 40.7187284460551)
POINT (-4.999999999999994 40.7

POINT (-4.01185318746852 40.5824187772507)
POINT (-4.01185318746852 40.61652229712559)
POINT (-4.01185318746852 40.65060841733385)
POINT (-4.01185318746852 40.68467713469507)
POINT (-4.01185318746852 40.7187284460551)
POINT (-4.01185318746852 40.75276234828597)
POINT (-4.01185318746852 40.78677883828593)
POINT (-4.01185318746852 40.82077791297934)
POINT (-4.01185318746852 40.85475956931668)
POINT (-4.01185318746852 40.88872380427453)
POINT (-4.01185318746852 40.92267061485544)
POINT (-4.01185318746852 40.95659999808807)
POINT (-4.01185318746852 40.99051195102698)


In [20]:
len(gridpoints)

690

In [4]:
with open('testout.csv', 'w') as of:
    of.write('lon;lat\n')
    for p in gridpoints:
        of.write('{:f};{:f}\n'.format(p.x, p.y))